# Elasticsearch Quick Start

<a target="_blank" href="https://colab.research.google.com/github/elasticsearch-labs/colab-notebooks-examples/blob/notebooks-guides/search/00-quick-start.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This interactive notebook will introduce you to the very basics of getting started with simple Elasticsearch queries, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
We'll run through getting the client up and running, indexing a small data set into Elasticsearch, and performing basic searches against your data.

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Select **Create deployment**

## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to install the `elasticsearch` Python client.

In [1]:
!pip install -qU elasticsearch sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.4 MB/s eta 0:00:00


Next we need to import the modules we need.

In [2]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen

and add the sentence transformer

In [3]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [4]:
# Create the client instance
client = Elasticsearch(
    cloud_id="My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQxYTU2YWQyMTU4N2M0NGQzOTMwOTMyZWI5ZmExZDhlOCRiNGZkMDBhYTNlZjI0ODdiYmU5OGQ5N2YyNTBlYWUyYw==",
    basic_auth=("elastic", "OdmjAw5cQGtTbvGqI3CTX18q")
)

If you're running Elasticsearch locally or self-managed, you can pass in the Elasticsearch host instead. [Read more](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later) on how to connect to Elasticsearch locally

Confirm that the client has connected with this test.

In [5]:
print(client.info())

{'name': 'instance-0000000000', 'cluster_name': '1a56ad21587c44d3930932eb9fa1d8e8', 'cluster_uuid': 'gX4zlwtlR4qhZpp1SPm4Yg', 'version': {'number': '8.8.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f8edfccba429b6477927a7c1ce1bc6729521305e', 'build_date': '2023-06-05T21:32:25.188464208Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## Index some test data

Our client is set up and connected to our Elastic deployment.
Now we need some data to test out the basics of Elasticsearch queries.
We'll use a small index of books with the following fields:

- `title`
- `authors`
- `publish_date`
- `num_reviews`
- `publisher`

### Create index

Let's create an Elasticsearch index with the correct mappings for our test data.

In [7]:
# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "authors": {"type": "keyword"},
            "summary": {"type": "text"},
            "publish_date": {"type": "date"},
            "num_reviews": {"type": "integer"},
            "publisher": {"type": "keyword"},
            "title_vector": { "type": "dense_vector", "dims": 384, "index": "true", "similarity": "dot_product" }
        }
    }
}

# Create the index
client.indices.create(index='book_index', body=mapping)


<ipython-input-7-d8ccee39cf78>:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(index='book_index', body=mapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'book_index'})

### Index test data

Run the following command to upload some test data, containing information about 10 popular programming books.

ℹ️ If you'd like to upload your own data from a URL, refer to the following [notebook](https://github.com/leemthompo/notebook-tests/blob/main/load-data-from-url.ipynb) for an example.

In [8]:
books = [
    {
        "title": "The Pragmatic Programmer: Your Journey to Mastery",
        "authors": ["andrew hunt", "david thomas"],
        "summary": "A guide to pragmatic programming for software engineers and developers",
        "publish_date": "2019-10-29",
        "num_reviews": 30,
        "publisher": "addison-wesley"
    },
    {
        "title": "Python Crash Course",
        "authors": ["eric matthes"],
        "summary": "A fast-paced, no-nonsense guide to programming in Python",
        "publish_date": "2019-05-03",
        "num_reviews": 42,
        "publisher": "no starch press"
    },
    {
        "title": "Artificial Intelligence: A Modern Approach",
        "authors": ["stuart russell", "peter norvig"],
        "summary": "Comprehensive introduction to the theory and practice of artificial intelligence",
        "publish_date": "2020-04-06",
        "num_reviews": 39,
        "publisher": "pearson"
    },
    {
        "title": "Clean Code: A Handbook of Agile Software Craftsmanship",
        "authors": ["robert c. martin"],
        "summary": "A guide to writing code that is easy to read, understand and maintain",
        "publish_date": "2008-08-11",
        "num_reviews": 55,
        "publisher": "prentice hall"
    },
    {
        "title": "You Don't Know JS: Up & Going",
        "authors": ["kyle simpson"],
        "summary": "Introduction to JavaScript and programming as a whole",
        "publish_date": "2015-03-27",
        "num_reviews": 36,
        "publisher": "oreilly"
    },
    {
        "title": "Eloquent JavaScript",
        "authors": ["marijn haverbeke"],
        "summary": "A modern introduction to programming",
        "publish_date": "2018-12-04",
        "num_reviews": 38,
        "publisher": "no starch press"
    },
    {
        "title": "Design Patterns: Elements of Reusable Object-Oriented Software",
        "authors": ["erich gamma", "richard helm", "ralph johnson", "john vlissides"],
        "summary": "Guide to design patterns that can be used in any object-oriented language",
        "publish_date": "1994-10-31",
        "num_reviews": 45,
        "publisher": "addison-wesley"
    },
    {
        "title": "The Clean Coder: A Code of Conduct for Professional Programmers",
        "authors": ["robert c. martin"],
        "summary": "A guide to professional conduct in the field of software engineering",
        "publish_date": "2011-05-13",
        "num_reviews": 20,
        "publisher": "prentice hall"
    },
    {
        "title": "JavaScript: The Good Parts",
        "authors": ["douglas crockford"],
        "summary": "A deep dive into the parts of JavaScript that are essential to writing maintainable code",
        "publish_date": "2008-05-15",
        "num_reviews": 51,
        "publisher": "oreilly"
    },
    {
        "title": "Introduction to the Theory of Computation",
        "authors": ["michael sipser"],
        "summary": "Introduction to the theory of computation and complexity theory",
        "publish_date": "2012-06-27",
        "num_reviews": 33,
        "publisher": "cengage learning"
    },
]

actions = []
for book in books:
    actions.append({"index": {"_index": "book_index"}})
    titleEmbedding = model.encode(book["title"]).tolist()
    book["title_vector"] = titleEmbedding
    actions.append(book)

client.bulk(index="book_index", operations=actions)


ObjectApiResponse({'took': 47, 'errors': False, 'items': [{'index': {'_index': 'book_index', '_id': 'eWsDIIkB6SgI-NN4Uquf', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'emsDIIkB6SgI-NN4Uquf', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'e2sDIIkB6SgI-NN4Uquf', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'fGsDIIkB6SgI-NN4Uquf', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'fWsDIIkB6SgI-NN4Uquf', '_version': 1, 'result': 'created', '_shards

## Aside: Pretty printing Elasticsearch responses

Your API calls will return hard-to-read nested JSON.
We'll create a little function called `pretty_response` to return nice, human-readable outputs from our examples.

In [ ]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nScore: {score}")
        print(pretty_output)

##Querying

Let's start by looking at simple queries which search for a particular value in a particular field.

In [11]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nScore: {score}")
        print(pretty_output)

response = client.search(index="book_index", body={
    "knn": {
      "field": "title_vector",
      "query_vector": model.encode("Best Project management books?"),
    "k": 10,
    "num_candidates": 100
    }
})

pretty_response(response)




<ipython-input-11-a3b0b73f7f11>:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="book_index", body={



ID: fGsDIIkB6SgI-NN4Uquf
Publication date: 2008-08-11
Title: Clean Code: A Handbook of Agile Software Craftsmanship
Summary: A guide to writing code that is easy to read, understand and maintain
Score: 0.6285683

ID: eWsDIIkB6SgI-NN4Uquf
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Score: 0.62295747

ID: gGsDIIkB6SgI-NN4Uquf
Publication date: 2011-05-13
Title: The Clean Coder: A Code of Conduct for Professional Programmers
Summary: A guide to professional conduct in the field of software engineering
Score: 0.5932041

ID: f2sDIIkB6SgI-NN4Uquf
Publication date: 1994-10-31
Title: Design Patterns: Elements of Reusable Object-Oriented Software
Summary: Guide to design patterns that can be used in any object-oriented language
Score: 0.5909667

ID: gmsDIIkB6SgI-NN4Uquf
Publication date: 2012-06-27
Title: Introduction to the Theory of Computation
Summary: Introduction to th

In [13]:
client.indices.delete(index="book_index")

ObjectApiResponse({'acknowledged': True})